In [11]:
# imports
import os
import cv2
from sklearn.ensemble import BaggingClassifier
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn
import sklearn.preprocessing as preproc
import numpy as np
import keras as k
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Flatten, Dense, Dropout, MaxPool2D
from keras.models import Sequential
#import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
#from time import time
import pandas as pd
from imblearn.over_sampling import SMOTENC
import tensorflow as tf
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
import math
import shutil
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.preprocessing import OneHotEncoder

In [3]:
image_path = '..\\..\\Untitled_Folder'
#### FABIO
# image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
csv_completo_2 = 'dataframe_completo_2.csv' 
#classifier = "cl_fit.h5"

In [4]:
# dal dataframe ne estrae le immagini in un array, serve nel batch
def load_images(array):
    images = list()
    #print(array.iloc[0])
    for index in range(0,len(array)):
        #print(index.type())
        img_id = array.iloc[index]
        elem = os.path.join(image_path, img_id)
        elem = elem + ".jpg"
        img = cv2.imread(elem)
        # TODO va fatto un reshape, non capisco perché
        images.append(np.asarray(img)) #########################################################################################
    return np.asarray(images)

In [5]:
# - carico in memoria il classificatore
# - istanzio l'encoder e lo fitto
dataset = pd.read_csv(csv_completo_2, encoding = "ISO-8859-1")
testset = pd.read_csv("test03.csv", encoding = "ISO-8859-1")

cl_batch = k.models.load_model("cl_batch.h5")
cl_fit = k.models.load_model("cl_fit.h5")

enc = OneHotEncoder(sparse=False)
y_train_shaped = np.array(dataset["dx"]).reshape(-1,1)
enc.fit(y_train_shaped)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=False)

In [7]:
# - carico in memoria le immagini test

test_img = load_images(testset["image_id"])

In [ ]:
# - predict del classificatore batch

predictions1 = cl_batch.predict(test_img)
pred_reverse1 = enc.inverse_transform(predictions1)
print(sklearn.metrics.classification_report(testset["dx"], pred_reverse1))
print('F1-score: '+ str(round(sklearn.metrics.f1_score(testset["dx"], pred_reverse1, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.66      0.31      0.43      2013
           1       0.60      0.63      0.61      1631
           2       0.50      0.56      0.53      2308
           3       0.70      0.81      0.75      2041
           4       0.58      0.63      0.60      2312
           5       0.75      0.83      0.79      2026
           6       0.95      0.94      0.94      2056

    accuracy                           0.67     14387
   macro avg       0.68      0.67      0.66     14387
weighted avg       0.67      0.67      0.66     14387

F1-score: 0.67


In [13]:
predictions2 = cl_fit.predict(test_img)
pred_reverse2 = enc.inverse_transform(predictions2)
print(sklearn.metrics.classification_report(testset["dx"], pred_reverse2))
print('F1-score: '+ str(round(sklearn.metrics.f1_score(testset["dx"], pred_reverse2, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.32      0.56      0.41      2013
           1       0.26      0.63      0.37      1631
           2       0.48      0.13      0.21      2308
           3       0.42      0.25      0.31      2041
           4       0.47      0.55      0.51      2312
           5       0.63      0.44      0.52      2026
           6       0.93      0.43      0.59      2056

    accuracy                           0.42     14387
   macro avg       0.50      0.43      0.42     14387
weighted avg       0.51      0.42      0.41     14387

F1-score: 0.42


In [ ]:
# DA CANCELLARE
trainset = pd.read_csv("train07.csv", encoding = "ISO-8859-1")
mass=int(len(trainset)*0.1)
samples = trainset.sample(n=mass)

print("loading images...")
train_img = load_images(samples["image_id"])

In [16]:

print("predicting...")
predictions3 = cl_fit.predict(train_img)
pred_reverse3 = enc.inverse_transform(predictions3)
print(sklearn.metrics.classification_report(samples["dx"], pred_reverse3))
print('F1-score: '+ str(round(sklearn.metrics.f1_score(samples["dx"], pred_reverse3, average='micro'),2)))


predicting...
              precision    recall  f1-score   support

           0       0.34      0.56      0.42       481
           1       0.29      0.63      0.40       411
           2       0.57      0.17      0.26       511
           3       0.43      0.30      0.35       450
           4       0.48      0.56      0.52       530
           5       0.66      0.48      0.56       488
           6       0.92      0.46      0.61       485

    accuracy                           0.45      3356
   macro avg       0.53      0.45      0.45      3356
weighted avg       0.53      0.45      0.45      3356

F1-score: 0.45


In [17]:
print("predicting...")
predictions4 = cl_batch.predict(train_img)
pred_reverse4 = enc.inverse_transform(predictions4)
print(sklearn.metrics.classification_report(samples["dx"], pred_reverse4))
print('F1-score: '+ str(round(sklearn.metrics.f1_score(samples["dx"], pred_reverse4, average='micro'),2)))

predicting...
              precision    recall  f1-score   support

           0       0.87      0.51      0.64       481
           1       0.80      0.78      0.79       411
           2       0.65      0.76      0.70       511
           3       0.76      0.88      0.81       450
           4       0.73      0.76      0.75       530
           5       0.84      0.88      0.86       488
           6       0.97      0.98      0.98       485

    accuracy                           0.79      3356
   macro avg       0.80      0.79      0.79      3356
weighted avg       0.80      0.79      0.79      3356

F1-score: 0.79
